# Data Cleaning

In [1]:
import numpy as np
import pandas as pd

### load data, drop unnecessary columns

In [223]:
recipes_ = pd.read_csv('../data_raw/recipes_0-63868.csv')
recipes_.drop('Unnamed: 0', axis=1, inplace=True)


### rename yield

In [227]:
recipes_.rename(columns=lambda x: x.replace('yield', 'yield_'), inplace=True)

### clean single-value columns
chef, difficulty, name, photo, rating, ratings, time_(cook, inactive, prep, total), url, yield

In [228]:
singles = ['chef', 'difficulty', 'name', 'photo', 'rating', 'ratings',
           'time_cook', 'time_inactive', 'time_prep', 'time_total', 'url', 'yield_']

In [229]:
def clean_item_columns(string):
    lst = []
    chars = ["[u'",'[u"',"']",'"]',"',"]
    for char in chars:
        string = string.replace(char,'')
    string = string.strip()
    string_list = string.split("u'")
    for string_ in string_list:
        string_ = string_.strip()
        lst.append(string_)
    if lst[0] == '[]':
        return ''
    else:
        return lst[0]

In [230]:
for column in recipes_[singles].columns:
    recipes_[column] = recipes_[column].map(clean_item_columns)

### make names all caps

In [235]:
def names_upper(string):
#     try:
    return string.upper()
#     except:
#         pass

In [236]:
recipes_['name'] = recipes_['name'].map(names_upper)

### clean list-value columns
categories, ingredients

In [238]:
def clean_ingredients(string):
    lst = []
    chars = ["[u'",'[u"',"']",'"]',"',",'",']
    for char in chars:
        string = string.replace(char,'')
    string = string.strip()
    string = string.lower()
    string_list = string.split("u'")
    for string_ in string_list:
        string_ = string_.strip()
        lst.append(string_)
    # to drop rows with no ingredients
    if len(lst) > 1:
        return lst
    else:
        return ''

In [239]:
def clean_categories(string):
    lst = []
    chars = ["[u'",'[u"',"']",'"]',"',",'",']
    for char in chars:
        string = string.replace(char,'')
    string = string.replace('u"Mother\'s',"u'Mothers")
    string = string.lower()
    string = string.strip()
    string_list = string.split("u'")
    for string_ in string_list:
        string_ = string_.strip()
        lst.append(string_)
    # to drop rows with no ingredients
    if len(lst) > 1:
        return lst
    else:
        return ''

In [240]:
recipes_['categories'] = recipes_.categories.map(clean_categories)
recipes_['ingredients'] = recipes_.ingredients.map(clean_ingredients)

### drop recipes with no ingredients or categories

In [242]:
recipes_[recipes_.ingredients == ''].shape

(3466, 15)

In [243]:
recipes_[recipes_.categories == ''].shape

(3613, 15)

In [244]:
recipes_[(recipes_.categories == '') & (recipes_.ingredients == '')].shape

(2065, 15)

In [245]:
recipes_ = recipes_[(recipes_.categories != '') & (recipes_.ingredients != '')]
recipes_ = recipes_[recipes_.ingredients != ''] 


In [246]:
recipes_.shape

(58854, 15)

In [247]:
recipes_[recipes_.categories == ''].shape

(0, 15)

### add column = ingredients_num

In [248]:
def numberize(lst):
    return float(len(lst))

In [249]:
recipes_['ingredients_num'] = recipes_['ingredients'].map(numberize)
# recipes_['categories_num'] = recipes_['categories'].map(numberize)

In [250]:
# recipes_.ingredients_num.value_counts()
recipes_.ingredients_num.isnull().sum()

0

### clean column = name

In [251]:
# drop blank values

In [252]:
recipes_.name = recipes_.name[recipes_.name != '']

In [253]:
recipes_.name.value_counts().head()

GUACAMOLE             35
TIRAMISU              23
FRIED CHICKEN         19
BASIC TOMATO SAUCE    19
SHEPHERD'S PIE        19
Name: name, dtype: int64

### clean column = difficulty

In [254]:
recipes_['difficulty'] = recipes_['difficulty'].map(lambda x: x if x != '' else 'Unknown')

In [255]:
recipes_.difficulty.value_counts()

Easy            36139
Unknown         11269
Intermediate    10619
Advanced          788
Expert             39
Name: difficulty, dtype: int64

### scale

In [256]:
recipes_['difficulty_scale'] = recipes_['difficulty'].map(lambda x: 1 if x == 'Easy'
                                                               else 2 if x == 'Intermediate'
                                                               else 3 if x == 'Advanced'
                                                               else 4 if x == 'Expert'
                                                               else 0)

In [257]:
recipes_.difficulty_scale.value_counts()

1    36139
0    11269
2    10619
3      788
4       39
Name: difficulty_scale, dtype: int64

### binarize - all difficulty values

In [258]:
recipes_['difficulty_easy'] = recipes_['difficulty'].map(lambda x: 1 if x == 'Easy' else 0)
recipes_['difficulty_intermediate'] = recipes_['difficulty'].map(lambda x: 1 if x == 'Intermediate' else 0)
recipes_['difficulty_advanced'] = recipes_['difficulty'].map(lambda x: 1 if x == 'Advanced' else 0)
recipes_['difficulty_expert'] = recipes_['difficulty'].map(lambda x: 1 if x == 'Expert' else 0)
recipes_['difficulty_unknown'] = recipes_['difficulty'].map(lambda x: 1 if x == 'Unknown' else 0)

In [259]:
recipes_.difficulty_intermediate.unique()

array([0, 1])

### clean column = time_cook, time_inactive

In [260]:
def clean_time(string):
    chars = ["<dd>",'</dd>']
    for char in chars:
        string = string.replace(char,'')
    return string

In [261]:
recipes_['time_cook'] = recipes_['time_cook'].map(clean_time)
recipes_['time_inactive'] = recipes_['time_inactive'].map(clean_time)

### clean column = directions
beautiful soup it

In [262]:
from bs4 import BeautifulSoup

In [263]:
def clean_directions(string):
    soup = BeautifulSoup(string, "lxml")
    text = soup.get_text()
    return text

In [264]:
def clean_directions(string):
    directions = ''
    soup = BeautifulSoup(string, "lxml")
    text = soup.get_text()
    for item in text:
        directions += item
    return directions

In [265]:
# still need to figure out how to get rid of some of the markup in this
# the output is unicode

In [266]:
recipes_['directions'] = recipes_['directions'].map(clean_directions)

### clean column = rating, ratings
convert to numeric values

In [267]:
# recipes_ = pd.read_csv('../data_clean/recipes_clean.csv')

In [268]:
def clean_ratings(string):
    string_list = string.split()
    try:
        return float(string_list[0])
    except:
        return 0

In [269]:
# def clean_ratings(num):
#     if num == 0:
#         return np.nan
#     else:
#         return num

In [270]:
recipes_['rating'] = recipes_['rating'].map(clean_ratings)
recipes_['ratings'] = recipes_['ratings'].map(clean_ratings)

In [271]:
# recipes_.to_csv('../data_clean/recipes_clean.csv')

### clean column = time_(prep, inactive, cook, total)
convert to minute values

In [272]:
def numberize_times(string):
    
    string_list = string.split()
    
    if len(string_list) == 2 and string_list[1] != 'hr':
        return float(string_list[0])
    elif len(string_list) == 2 and string_list[1] == 'hr':
        return 60.*(float(string_list[0]))
    elif len(string_list) > 2:
        return 60.*(float(string_list[0])) + float(string_list[2])
    else:
        return 0

In [273]:
recipes_['time_cook'] = recipes_['time_cook'].map(numberize_times)
recipes_['time_prep'] = recipes_['time_prep'].map(numberize_times)
recipes_['time_inactive'] = recipes_['time_inactive'].map(numberize_times)
recipes_['time_total'] = recipes_['time_total'].map(numberize_times)

### add column = photo_binary

In [274]:
recipes_['photo_binary'] = recipes_['photo'].map(lambda x: 0 if x == '' else 1)

In [275]:
recipes_['photo_binary'].value_counts()

1    31674
0    27180
Name: photo_binary, dtype: int64

### create 3 category columns and append each category-list item

In [276]:
def category_column_1(lst):
    try:
        return lst[0]
    except:
        return ''

In [277]:
def category_column_2(lst):
    try:
        return lst[1]
    except:
        return ''

In [278]:
def category_column_3(lst):
    try:
        return lst[2]
    except:
        return ''

In [279]:
recipes_['category_1'] = recipes_['categories'].map(category_column_1)
recipes_['category_2'] = recipes_['categories'].map(category_column_2)
recipes_['category_3'] = recipes_['categories'].map(category_column_3)

### create numerical category columns

In [280]:
unique_categories = []
def category_list(lst):
    for item in lst:
        if item not in unique_categories:
            unique_categories.append(item)
        else:
            pass

In [281]:
recipes_.categories.map(category_list)

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
         ... 
63836    None
63837    None
63838    None
63839    None
63840    None
63841    None
63842    None
63844    None
63845    None
63846    None
63847    None
63848    None
63849    None
63850    None
63851    None
63852    None
63853    None
63854    None
63855    None
63856    None
63857    None
63858    None
63859    None
63860    None
63861    None
63862    None
63863    None
63864    None
63866    None
63867    None
Name: categories, dtype: object

In [282]:
print len(unique_categories)

424


In [283]:
# category_range = []
# for i in range(1,(len(unique_categories)+1)):
#     category_range.append(i)
# categories_numerical = zip(category_range, unique_categories)
# categories_numerical[:5]

In [284]:
unique_categories.index('apple')

2

In [285]:
def categories_numerical_values(string):
    try:
        return float(unique_categories.index(string)+1)
    except:
        return 0

In [286]:
recipes_['category_1_num'] = recipes_['category_1'].map(categories_numerical_values)
recipes_['category_2_num'] = recipes_['category_2'].map(categories_numerical_values)
recipes_['category_3_num'] = recipes_['category_3'].map(categories_numerical_values)

In [287]:
# category_dict['category'] = [category for category in unique_categories]

In [288]:
category_df = pd.DataFrame(unique_categories)

In [289]:
category_df.reset_index(inplace=True)

In [290]:
def reset_indx(num):
    return num+1

category_df['index'] = category_df['index'].map(reset_indx)

In [291]:
category_df.columns

Index([u'index', 0], dtype='object')

In [292]:
category_df.head()

,index,0
0,1,bourbon
1,2,dessert
2,3,apple
3,4,side dish
4,5,sauce


In [293]:
category_df.to_csv('../data_raw/categories_no_nan.csv')

### get rid of unicode in directions

In [294]:
def no_more_unicode(uni_code):
    return str(uni_code)

In [353]:
recipes_['directions'] = recipes_['directions'].map(no_more_unicode)

### get rid of unicode characters in directions

In [354]:
def no_more_unicode_characters(string):
    chars = ["[u'",'[u"',"']",'"]',"',",'",',"',",'u"',"u'","\'"]
    for char in chars:
        string = string.replace(char,'')
    string = string.strip()
    return string

In [355]:
recipes_['directions'] = recipes_['directions'].map(no_more_unicode_characters)

### add a column for length of instructions (characters)

In [298]:
def directions_length(string):
    return float(len(string))

In [356]:
recipes_['directions_len'] = recipes_['directions'].map(directions_length)

### clean up the chef column as best as possible

In [409]:
tmp = recipes_

In [447]:
def clean_chef(string):
    string = string.lower()
    chars = ['recipe courtesy of ', 'courtesy of', 'for food network magazine', 'all rights reserved',
             '1','2','3','4','5','6','7','8','9','0', '.', ',', 'from', 'copyright', 'recipe', 'by']
    for char in chars:
        string = string.replace(char,'')
    string = string.strip()
    return string
    

In [448]:
tmp['chef'] = tmp['chef'].map(clean_chef)

### check results on everything

In [459]:
# recipes = recipes_

random = int(tmp.shape[0]*np.random.rand())

# not_directions = [col for col in recipes_.columns if col != 'directions']

# for column in recipes[not_directions].columns:
for column in tmp.columns:
    print "%s: " % column, '\n', type(tmp[column][random]), '\n', tmp[column][random], '\n'

categories:  
<type 'list'> 
['eggs', 'breakfast', 'poaching'] 

chef:  
<type 'str'> 
cafe 

difficulty:  
<type 'str'> 
Intermediate 

directions:  
<type 'str'> 
Preheat the oven to 350 degrees F. Make sure all ingredients are at room temperature before you start. In a large bowl place the potatoes, olive oil, all-purpose seasoning and green pepper. Mix well until all ingredients are combined, then add salt, to taste. Transfer the potatoes to a baking pan and place in the preheated oven for about 20 minutes or until potatoes are soft. Set aside. While the potatoes are in the oven, heat a large skillet over medium-high heat. Add the Mexican chorizo and saute until it separates and has a crumbled consistency, about 8 minutes. Then set aside.  For the spicy hollandaise sauce: Set an stainless steel bowl over a pot with barely simmering water. The bowl should fit the pot but barely touching the water. Place the butter into the bowl and whisk until it is melted. Add the egg yolks, 1 at a